In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
max_steps = 1001
image_num = 3000

sess = tf.Session()

embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None,784], name='x_input')
    y = tf.placeholder(tf.float32, [None,10], name='y_input')
    
with tf.name_scope('input_reshape'):
    image_shape_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shape_input, 10)
   
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        w = tf.Variable(tf.zeros([784,10]), name='w')
        variable_summaries(w)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,w)+b
        variable_summaries(wx_plus_b)
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
        
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.01).minimize(loss)
    
with tf.name_scope('accurary'):
    #结果存放在布尔表中
    with tf.name_scope('correct_predict'):
        correct_predict = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accurary'):
        accurary = tf.reduce_mean(tf.cast(correct_predict,tf.float32))
        tf.summary.scalar('accurary',accurary)
        
#产生metadata文件
if tf.gfile.Exists('./projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively('./projector/projector/metadata.tsv')
    
with open('./projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.arg_max(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
        
#合并所有summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter('./projector/projector',sess.graph)
saver = tf.train.Saver()

# 定义配置文件
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = './projector/projector/metadata.tsv'
embed.sprite.image_path = './projector/data/mnist_10k_sprite.png'

# 切分图片
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer, config)
    
    
for i in range(max_steps):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_  = sess.run([merged, train_step], feed_dict={x:batch_xs,y:batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.iamges, y:mnist.test.labels})
        print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
        
saver.save(sess, './projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



FileNotFoundError: ignored